In [ ]:
from fastai.vision.all import *
from fastai.learner import *
from fastai.data.all import *
from fastai.callback.tracker import SaveModelCallback
import pandas as pd
import matplotlib.pyplot as plt
from pathlib2 import Path
import numpy as np
import random
from torch.nn import MSELoss

In [ ]:
%%time

df = pd.read_csv('data/examples.csv')
df.shape

In [ ]:
%%time

uniq_fns = np.unique(df.target_fn.values)
fn2feature = {}
for fn in uniq_fns:
    ary = pd.read_pickle(f'data/examples/{fn}.pkl')
    fn2feature[fn] = ary

pd.to_pickle(fn2feature, 'data/fn2feature.pkl')

In [ ]:
%%time
fn2features = pd.read_pickle('data/fn2feature.pkl')

In [ ]:
%%time
fn2feature_length = {}

for fn, feature in fn2features.items():
    fn2feature_length[fn] = feature.shape[0]

In [ ]:
%%time

df['source_length'] = df.source_fn.apply(lambda fn: fn2feature_length[fn])
df['target_length'] = df.target_fn.apply(lambda fn: fn2feature_length[fn])

In [ ]:
df.to_csv('data/examples_with_length.csv', index=False)

In [ ]:
((df.source_length < 70) & (df.target_length < 70)).mean()

In [ ]:
df.source_word.nunique()

In [ ]:
df[((df.source_length < 70) & (df.target_length < 70))].source_word.nunique()

In [ ]:
df[((df.source_length < 70) & (df.target_length < 70) & (df.source_length > 25))].drop_duplicates(subset='source_fn').sort_values('source_length', ascending=True).head(20)

In [ ]:
from IPython.lib.display import Audio

In [ ]:
Audio(filename='data/examples_audio/8673ec44a32140a09272133796bf7126.wav')

In [ ]:
df[((df.source_length < 70) & (df.target_length < 70) & (df.source_length > 20))].source_word.nunique()